<a href="https://colab.research.google.com/github/jimkc/Scripts/blob/main/Collect_YouTubeComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Output as Pandas DataFrame

In [26]:
import googleapiclient.discovery
import pandas as pd
import re

# Function to extract video ID from YouTube URL
def extract_video_id(url):
    # Regular expression for extracting the video ID
    regex = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    matches = re.search(regex, url)
    if matches:
        return matches.group(1)
    else:
        raise ValueError("Unable to extract Video ID from the provided URL.")

# Sample YouTube URL (User should replace this with the actual URL)
youtube_url = ""

# Set the parameter of credential and how many comments to get below
DEVELOPER_KEY = ""
COMMENTS_TO_GET = 100
# Extracting the video ID from the URL
VIDEO_ID = extract_video_id(youtube_url)

api_service_name = "youtube"
api_version = "v3"

def get_all_comments(youtube, video_id, max_comments):
  """
  Retrieves all comments for a given video ID using pagination.

  Args:
      youtube: The authorized YouTube service object.
      video_id: The ID of the video for which to retrieve comments.

  Returns:
      A list containing all retrieved comments (up to quota limits).
  """
  all_comments = []
  page_token = None
  # Retrieve with pagination
  while len(all_comments) < max_comments:
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=min(max_comments, 1000),
        pageToken=page_token,
    )
    response = request.execute()

    # Get comments from current page
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        all_comments.append([
            comment['authorDisplayName'],
            comment['publishedAt'],
            comment['updatedAt'],
            comment['likeCount'],
            comment['textDisplay']
        ])

    # Check for next page token
    page_token = response.get("nextPageToken")
    if not page_token:
      break

  return all_comments

# Make the calls
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)
comments = get_all_comments(youtube, VIDEO_ID, COMMENTS_TO_GET)
print(f"Retrieved {len(comments)} comments")


df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

# Uncomment if you want to display the results, keep commented to speed up if the COMMENTS_TO_GET is large
# df.head(COMMENTS_TO_GET)


Retrieved 199 comments


In [18]:
import os

# Define the directory path
directory_path = '/mnt/data'

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [19]:
df.to_csv('/mnt/data/my_dataframe.csv', index=False)